# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
!pip install ucimlrepo
!pip install azure-ai-ml

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
# choose a name for experiment
experiment_name = 'parkinson_exp'

experiment=Experiment(ws, experiment_name)
project_folder = './pipeline-parkinson-project'

quick-starts-ws-259725
aml-quickstarts-259725
westeurope
610d6e37-4747-4a20-80eb-3aad70a55f43


In [85]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 
  
# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features 
y = parkinsons_telemonitoring.data.targets 
  
# metadata 
print(parkinsons_telemonitoring.metadata) 
  
# variable information 
print(parkinsons_telemonitoring.variables) 


{'uci_id': 189, 'name': 'Parkinsons Telemonitoring', 'repository_url': 'https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring', 'data_url': 'https://archive.ics.uci.edu/static/public/189/data.csv', 'abstract': "Oxford Parkinson's Disease Telemonitoring Dataset", 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Tabular'], 'num_instances': 5875, 'num_features': 19, 'feature_types': ['Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['motor_UPDRS', 'total_UPDRS'], 'index_col': ['subject#'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5ZS3N', 'creators': ['Athanasios Tsanas', 'Max Little'], 'intro_paper': {'title': "Accurate Telemonitoring of Parkinson's Disease Progression by Noninvasive Speech Tests", 'authors': 'A. Tsanas, Max A. Little, P. McSharry, L. Ramig', 'published_in': 'IEEE Transactions on Biomedical Engineering', 

In [86]:
X.reset_index(inplace=True)
y.reset_index(inplace=True)

#df.to_csv("parkinson_data.csv",index=False)

In [87]:
df=pd.merge(X,y,on='index')

df.drop(columns=['index','motor_UPDRS'],inplace=True)
# Since here we have two variables that can be target variable , focusing on one & removing the other

In [95]:
df.columns
df.to_csv("parkinson_data_v2.csv",index=False)
# Saving the above file to a csv so as to upload as a Dataset using Upload files locally from ML Studio

#### Since Seems like regsitering a dataframe directly is something not possible in sdk v2 , I would directly create the dataset using stuido & access it 

In [42]:
# found = False
# key = "parkinson_data"
# description_text = "Oxford Parkinson's Disease Telemonitoring Dataset taken from UCI"

# if key in ws.datasets.keys(): 
#         found = True
#         dataset = ws.datasets[key] 

# if not found:
#         # Create AML Dataset and register it into Workspace
#         dataset = Dataset.Tabular.from_delimited_files(example_data)        
#         #Register Dataset in Workspace
#         dataset = dataset.register(workspace=ws,
#                                    name=key,
#                                    description=description_text)


# df = dataset.to_pandas_dataframe()
# df.describe()

In [4]:
key='parkinson-data-v2'
if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 


## Validating a compute if not , creating one

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'r2_score'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="total_UPDRS",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)
                            


In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
parkinson_exp,AutoML_b17b0511-213f-4a47-8f1c-d07a4bf23e89,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)
remote_run

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
parkinson_exp,AutoML_b17b0511-213f-4a47-8f1c-d07a4bf23e89,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

Experiment,Id,Type,Status,Details Page,Docs Page
parkinson_exp,AutoML_b17b0511-213f-4a47-8f1c-d07a4bf23e89,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run,fitted_model=remote_run.get_output()
remote_run.get_metrics()

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection',
  'BestRunExplainModel',
  'ModelExplanationDataSetSetup',
  'PickSurrogateModel',
  'EngineeredFeatureExplanations',
  'EngineeredFeatureExplanations',
  'RawFeaturesExplanations',
  'RawFeaturesExplanations',
  'BestRunExplainModel'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.',
  'Best run model explanations started',
  'Model explanations data setup completed',
  'Choosing LightGBM as the surrogate model for explanations',
  'Computation of engineered features started',
  'Computation of engineered features completed',
  'Computation of ra

In [10]:
print(best_run)

Run(Experiment: parkinson_exp,
Id: AutoML_b17b0511-213f-4a47-8f1c-d07a4bf23e89_39,
Type: azureml.scriptrun,
Status: Completed)


In [11]:
print(fitted_model)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             StackEnsembleRegressor(base_learners=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostregressor', XGBoostRegressor(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostregressor', XGBoostRegressor(booster='gbtree', colsample_bytree=1

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
inference_folder_name = 'inference'
automl_model = os.path.join(inference_folder_name, 'model.pkl')
score_script = os.path.join(inference_folder_name, 'score.py')
conda_env = os.path.join(inference_folder_name, 'conda_env.yml')
best_run.download_file('outputs/model.pkl', automl_model)
best_run.download_file('outputs/scoring_file_v_1_0_0.py', score_script)
best_run.download_file('outputs/conda_env_v_1_0_0.yml', conda_env)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
import sklearn
from azureml.core import Model 
model = Model.register(
    workspace=ws,
    model_name=best_run.properties['model_name'],
    model_path=automl_model,
    model_framework=Model.Framework.SCIKITLEARN,
    model_framework_version=sklearn.__version__,
    description='Auto ML model for predicting total updrs Parkinson  '
    )

print('Name:', model.name)
print('Version:', model.version)

Registering model AutoMLb17b0511239
Name: AutoMLb17b0511239
Version: 1


In [14]:
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.model import InferenceConfig
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.webservice import AciWebservice
env = Environment.from_conda_specification(name="env", file_path=conda_env)
inference_conf = InferenceConfig(entry_script=score_script, environment=env)

# Display the environment file
with open(conda_env, 'r') as file:
    env_file = file.read()
    print(env_file)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.8 and later.
- python=3.8.17

- pip:
  - azureml-train-automl-runtime==1.52.0
  - inference-schema
  - azureml-interpret==1.52.0
  - azureml-defaults==1.52.0
- numpy==1.22.3
- pandas==1.1.5
- scikit-learn==0.22.1
- py-xgboost==1.3.3
- fbprophet==0.7.1
- holidays==0.10.3
- psutil==5.9.5
channels:
- anaconda
- conda-forge



TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
deployment_conf = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predicting total updrs for parkinson diseases',
    enable_app_insights=True)

# Deploy model as webservice using Azure Container Instance (ACI)
service_name = "parkinson-predict"

service = Model.deploy(
    workspace=ws,
    name=service_name, 
    models=[model], 
    inference_config=inference_conf, 
    deployment_config=deployment_conf, 
    overwrite=True)

service.wait_for_deployment(show_output=True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-05-26 20:16:42+00:00 Creating Container Registry if not exists.
2024-05-26 20:16:42+00:00 Registering the environment.
2024-05-26 20:16:44+00:00 Use the existing image.
2024-05-26 20:16:44+00:00 Generating deployment configuration.
2024-05-26 20:16:44+00:00 Submitting deployment to compute.
2024-05-26 20:16:48+00:00 Checking the status of deployment parkinson-predict..
2024-05-26 20:20:18+00:00 Checking the status of inference endpoint parkinson-predict.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [23]:
ws = Workspace.from_config()
key='parkinson-data-v2'
ds = ws.datasets[key]
df=ds.to_pandas_dataframe()
df.drop(columns='total_UPDRS').head(3).to_json(orient='records')

'[{"age":72,"test_time":5.6431,"Jitter(%)":0.00662,"Jitter(Abs)":0.0000338,"Jitter:RAP":0.00401,"Jitter:PPQ5":0.00317,"Jitter:DDP":0.01204,"Shimmer":0.02565,"Shimmer(dB)":0.23,"Shimmer:APQ3":0.01438,"Shimmer:APQ5":0.01309,"Shimmer:APQ11":0.01662,"Shimmer:DDA":0.04314,"NHR":0.01429,"HNR":21.64,"RPDE":0.41888,"DFA":0.54842,"PPE":0.16006,"sex":0},{"age":72,"test_time":12.666,"Jitter(%)":0.003,"Jitter(Abs)":0.0000168,"Jitter:RAP":0.00132,"Jitter:PPQ5":0.0015,"Jitter:DDP":0.00395,"Shimmer":0.02024,"Shimmer(dB)":0.179,"Shimmer:APQ3":0.00994,"Shimmer:APQ5":0.01072,"Shimmer:APQ11":0.01689,"Shimmer:DDA":0.02982,"NHR":0.011112,"HNR":27.183,"RPDE":0.43493,"DFA":0.56477,"PPE":0.1081,"sex":0},{"age":72,"test_time":19.681,"Jitter(%)":0.00481,"Jitter(Abs)":0.0000246,"Jitter:RAP":0.00205,"Jitter:PPQ5":0.00208,"Jitter:DDP":0.00616,"Shimmer":0.01675,"Shimmer(dB)":0.181,"Shimmer:APQ3":0.00734,"Shimmer:APQ5":0.00844,"Shimmer:APQ11":0.01458,"Shimmer:DDA":0.02202,"NHR":0.02022,"HNR":23.047,"RPDE":0.46222,"D

In [24]:
import json
import requests

test_data = json.dumps({
    "data": [
       {"age":72,"test_time":5.6431,"Jitter(%)":0.00662,"Jitter(Abs)":0.0000338,"Jitter:RAP":0.00401,"Jitter:PPQ5":0.00317,"Jitter:DDP":0.01204,"Shimmer":0.02565,"Shimmer(dB)":0.23,"Shimmer:APQ3":0.01438,"Shimmer:APQ5":0.01309,"Shimmer:APQ11":0.01662,"Shimmer:DDA":0.04314,"NHR":0.01429,"HNR":21.64,"RPDE":0.41888,"DFA":0.54842,"PPE":0.16006,"sex":0},{"age":72,"test_time":12.666,"Jitter(%)":0.003,"Jitter(Abs)":0.0000168,"Jitter:RAP":0.00132,"Jitter:PPQ5":0.0015,"Jitter:DDP":0.00395,"Shimmer":0.02024,"Shimmer(dB)":0.179,"Shimmer:APQ3":0.00994,"Shimmer:APQ5":0.01072,"Shimmer:APQ11":0.01689,"Shimmer:DDA":0.02982,"NHR":0.011112,"HNR":27.183,"RPDE":0.43493,"DFA":0.56477,"PPE":0.1081,"sex":0},{"age":72,"test_time":19.681,"Jitter(%)":0.00481,"Jitter(Abs)":0.0000246,"Jitter:RAP":0.00205,"Jitter:PPQ5":0.00208,"Jitter:DDP":0.00616,"Shimmer":0.01675,"Shimmer(dB)":0.181,"Shimmer:APQ3":0.00734,"Shimmer:APQ5":0.00844,"Shimmer:APQ11":0.01458,"Shimmer:DDA":0.02202,"NHR":0.02022,"HNR":23.047,"RPDE":0.46222,"DFA":0.54405,"PPE":0.21014,"sex":0}]

           })

response = requests.post(
    service.scoring_uri, 
    data=test_data, 
    headers={'Content-Type':'application/json'})

print("Results:", response.json())

Results: {"result": [34.255789107026445, 35.25498673949362, 35.357035157967275]}


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
